In [2]:
import random

# Define Personas and their Purchase Habits
personas = {
    "The Confident Boomer":         {"percentage": 35, "purchases": {"Espresso Drink": 0.4, "Pour Over Coffee": 0.5, "Tea": 0.1, "Sparkling Tea Drink": 0.2, "Pastry": 0.4, "Sandwich": 0.5}},
    "The Affluent Professional":    {"percentage": 20, "purchases": {"Espresso Drink": 0.6, "Pour Over Coffee": 0.3, "Pastry": 0.6, "Sandwich": 0.6, "Sparkling Tea Drink": 0.3, "Bubble Tea": 0.2}},
    "The Family":                   {"percentage": 30, "purchases": {"Bubble Tea": 0.8, "Bubble Tea Upgrades": 0.2, "Kids Drink": 0.5, "Pastry": 0.3, "Sandwich": 0.3, "Popcorn Chicken": 0.4, "Garlic Fries": 0.5, "Waffles": 0.4}},
    "The Socializer":               {"percentage": 8,  "purchases": {"Bubble Tea": 0.5, "Bubble Tea Upgrades": 0.2, "Sparkling Tea Drink": 0.5, "Tea": 0.2, "Garlic Fries": 0.5, "Popcorn Chicken": 0.6, "Waffles": 0.3}},
    "The Health-Conscious":         {"percentage": 7,  "purchases": {"Tea": 0.2, "Espresso Drink": 0.2, "Pour Over Coffee": 0.3, "Sparkling Tea Drink": 0.7, "Sandwich": 0.5, "Pastry": 0.2}},
}

# New: Define average prices for each menu item
prices = {
    "Espresso Drink": 6.5,
    "Pour Over Coffee": 5.0,
    "Pastry": 5,
    "Sandwich": 11.0,
    "Bubble Tea": 6.5,
    "Bubble Tea Upgrades": .75,
    "Tea": 5.0,
    "Sparkling Tea Drink": 5.0,
    "Kids Drink": 4.0,
    "Waffles": 5.0,
    "Popcorn Chicken": 7.50,
    "Garlic Fries": 6.50,
}

costs = {
    "Espresso Drink": 2.0,
    "Pour Over Coffee": 1.5,
    "Pastry": 2.0,
    "Sandwich": 3.0,
    "Bubble Tea": 2.5,
    "Bubble Tea Upgrades": .05,
    "Tea": 0.25,
    "Sparkling Tea Drink": 1.5,
    "Kids Drink": 1.0,
    "Waffles": 1.0,
    "Popcorn Chicken": 1.46,
    "Garlic Fries": 1.00,
}

item_relations = {
    "Espresso Drink": {"Pastry": 0.2},
    "Pour Over Coffee": {"Pastry": 0.2},
    "Pastry": {"Espresso Drink": 0.1, "Pour Over Coffee": 0.1, "Tea": 0.05, "Bubble Tea": 0.1},
    "Sandwich": {"Espresso Drink": 0.05, "Sparkling Tea Drink": 0.1, "Bubble Tea": 0.1},
    "Bubble Tea": {"Waffles": 0.3},
    "Tea": {"Pastry": 0.1},
    "Sparkling Tea Drink": {"Sandwich": 0.05},
    "Kids Drink": {"Pastry": 0.1},
    "Waffles": {"Bubble Tea": 0.3, "Tea": 0.1},
    "Popcorn Chicken": {"Bubble Tea": 0.2,},
    "Garlic Fries": {"Bubble Tea": 0.1,},
}

sample_orders_by_persona = {persona: [] for persona in personas.keys()} 

def adjust_probability(item, selected_items):
    adjustment = 0
    for related_item, effect in item_relations.get(item, {}).items():
        if related_item in selected_items:
            adjustment += effect
    return adjustment

def calculate_purchases_and_revenue(estimated_total_customers):
    total_purchases = {item: 0 for item in prices.keys()}
    total_revenue = {item: 0 for item in prices.keys()}
    total_costs = {item: 0 for item in costs.keys()}  
    total_profit = {item: 0 for item in prices.keys()}  
    
    for persona, data in personas.items():
        num_customers_in_persona = (data['percentage'] / 100) * estimated_total_customers
        
        for _ in range(int(num_customers_in_persona)):
            selected_items = []
            
            for item, probability in data['purchases'].items():
                adjusted_probability = probability + adjust_probability(item, selected_items)
                if random.random() < adjusted_probability:
                    total_purchases[item] += 1
                    total_revenue[item] += prices[item]
                    total_costs[item] += costs[item]  
                    selected_items.append(item)
                    
            # Enhanced Fallback Mechanism
            if not selected_items:
                items = list(data['purchases'].keys())
                probabilities = [data['purchases'][item] for item in items]
                item = random.choices(items, weights=probabilities, k=1)[0]
                
                total_purchases[item] += 1
                total_revenue[item] += prices[item]
                total_costs[item] += costs[item]
                selected_items.append(item)
            
            if len(sample_orders_by_persona[persona]) < 5:
                sample_orders_by_persona[persona].append(selected_items)
    
    # Calculate total profit for each item
    for item in total_profit.keys():
        total_profit[item] = total_revenue[item] - total_costs[item]
    
    total_revenue_sum = sum(total_revenue.values())
    total_costs_sum = sum(total_costs.values())
    total_profit_sum = sum(total_profit.values())
    
    return total_purchases, total_revenue, total_costs, total_profit, total_revenue_sum, total_costs_sum, total_profit_sum, sample_orders_by_persona

def calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations):
    # Initialize accumulators for total purchases, revenue, costs, and profit for each item
    item_purchases_sums = {item: 0 for item in prices.keys()}
    simulations_revenue = []
    simulations_costs = []
    simulations_profit = []
    
    for i in range(num_simulations):
        # Run the simulation and unpack all the additional return values properly
        total_purchases, total_revenue, total_costs, total_profit, total_revenue_sum, total_costs_sum, total_profit_sum, _ = calculate_purchases_and_revenue(estimated_total_customers)
        
        # Accumulate total purchases, revenue, costs, and profit for each item
        for item in total_purchases.keys():
            item_purchases_sums[item] += total_purchases[item]
        simulations_revenue.append(total_revenue_sum)
        simulations_costs.append(total_costs_sum)
        simulations_profit.append(total_profit_sum)
        
        # Optionally log completion of each simulation
        if (i+1) % 1000 == 0 or i == num_simulations - 1:
            print(f"Simulation {i+1}/{num_simulations} completed.")
    
    # Calculate the average total purchases, revenue, costs, and profit across all simulations
    avg_item_purchases = {item: total / num_simulations for item, total in item_purchases_sums.items()}
    avg_total_revenue = sum(simulations_revenue) / num_simulations
    avg_total_costs = sum(simulations_costs) / num_simulations
    avg_total_profit = sum(simulations_profit) / num_simulations
    
    return avg_item_purchases, avg_total_revenue, avg_total_costs, avg_total_profit

estimated_total_customers = 2500
num_simulations = 1000
# Adjusted call to the Monte Carlo simulation function
avg_item_purchases, avg_total_revenue, avg_total_costs, avg_total_profit = calculate_purchases_and_revenue_monte_carlo(estimated_total_customers, num_simulations)

avg_total_revenue_per_year = avg_total_revenue * 12

# Displaying the results
print("Average Purchases for Each Menu Item:")
for item, avg_purchases in avg_item_purchases.items():
    print(f"{item}: {avg_purchases:.2f}")
print(f"\nAverage Total Revenue per Month: ${avg_total_revenue:.2f}")
print(f"Average Total Costs per Month: ${avg_total_costs:.2f}")
print(f"Average Gross Margin (Revenue - COGS) per Month: ${avg_total_profit:.2f}")

print(f"\nAverage Total Revenue per Year: ${avg_total_revenue_per_year:.2f}")

print('--------- Sample Orders --------- ')

# Iterate through each persona to display their sample orders
for persona, orders in sample_orders_by_persona.items():
    print(f"\nSample Orders for {persona}:")
    for order_index, order in enumerate(orders, start=1):
        print(f"  Order {order_index}: {order}")


Simulation 1000/1000 completed.
Average Purchases for Each Menu Item:
Espresso Drink: 699.06
Pour Over Coffee: 656.45
Pastry: 1124.30
Sandwich: 1192.43
Bubble Tea: 803.46
Bubble Tea Upgrades: 190.46
Tea: 166.38
Sparkling Tea Drink: 573.92
Kids Drink: 376.12
Waffles: 575.59
Popcorn Chicken: 560.69
Garlic Fries: 547.27

Average Total Revenue per Month: $47776.05
Average Total Costs per Month: $13446.92
Average Gross Margin (Revenue - COGS) per Month: $34329.13

Average Total Revenue per Year: $573312.60
--------- Sample Orders --------- 

Sample Orders for The Confident Boomer:
  Order 1: ['Sandwich']
  Order 2: ['Pour Over Coffee', 'Sandwich']
  Order 3: ['Espresso Drink', 'Pastry']
  Order 4: ['Pour Over Coffee', 'Sparkling Tea Drink', 'Sandwich']
  Order 5: ['Pour Over Coffee', 'Sparkling Tea Drink', 'Pastry', 'Sandwich']

Sample Orders for The Affluent Professional:
  Order 1: ['Pastry', 'Sandwich']
  Order 2: ['Espresso Drink', 'Pour Over Coffee', 'Pastry', 'Sandwich', 'Sparkling Te